In [2]:
from __future__ import division
import pandas as pd
import warnings
import seaborn as sns
import matplotlib.pyplot as plt
from pylab import rcParams
%matplotlib inline
import json

warnings.filterwarnings("ignore", category=DeprecationWarning)
sns.set_style("whitegrid")
sns.set_context("poster")
rcParams['figure.figsize'] = 20,5

In [9]:
cd ../app

/media/aok1425/1f5fa7c1-6532-4a01-8076-91f01ca10cae/PycharmProjects/galvanize/311-prediction-times/app


In [12]:
from models import make_q1_map_json

In [18]:
tract_and_block_groups = []
issues_by_year_per_1000 = []
entries = {}

In [ ]:
data['features'][]

In [32]:
for entry in data['features']:
    entry = entry['properties']
    d = {}
    
    for k in entry:
        if 'total_issues' in k and 'per_1000' in k and '2017' not in k:
            try:
                d[k] = int(entry[k])
            except TypeError:
                d[k] = 0

    id_ = entry['tract_and_block_group']
    entries[id_] = d

## Let's check

In [33]:
df = pd.DataFrame(entries, dtype=int).T
print df.shape
df.head()

(558, 7)


,total_issues_2011_per_1000,total_issues_2012_per_1000,total_issues_2013_per_1000,total_issues_2014_per_1000,total_issues_2015_per_1000,total_issues_2016_per_1000,total_issues_all_years_per_1000
0001001,81.0,151.0,178.0,238.0,370.0,200.0,1218.0
0001002,44.0,97.0,139.0,141.0,193.0,166.0,780.0
0001003,94.0,191.0,287.0,215.0,301.0,353.0,1444.0
0001004,37.0,57.0,135.0,128.0,209.0,182.0,753.0
0002011,51.0,118.0,132.0,127.0,216.0,250.0,894.0


In [34]:
df['total_issues_2014_per_1000'].describe()

count     558.000000
mean      162.867384
std       405.401922
min         0.000000
25%        87.000000
50%       121.500000
75%       162.750000
max      7667.000000
Name: total_issues_2014_per_1000, dtype: float64

The max is way bigger than the 75th percentile. Let's look at this range.

In [38]:
df['total_issues_2014_per_1000'].sort_values(ascending=False).head(10)

9807001    7667.0
9818001    4273.0
0303003    3205.0
0701018    2248.0
9811003    1231.0
1101033     462.0
0102034     460.0
0512001     437.0
0701015     434.0
0701011     424.0
Name: total_issues_2014_per_1000, dtype: float64

For 2014, the top 5 seem to be outliers. Where are they?

- 9807001 is Stony Brook Reservation
- 9818001 is the Fens river
- 0303003 is Downtown Crossing/Park Street/Haymarket--no one lives there. 1,500 pop.
- 0701018 is South Station. 210 pop.
- 9811003 is Forest Hills Cemetery. 13 pop.

Do we see the same 5 block groups for other years too?

In [40]:
df['total_issues_2013_per_1000'].sort_values(ascending=False).head(10)

9807001    7778.0
9818001    4091.0
0303003    2615.0
0701018    1762.0
9811003     923.0
0102034     577.0
0403005     482.0
1101033     415.0
0402002     411.0
0107021     411.0
Name: total_issues_2013_per_1000, dtype: float64

In [41]:
df['total_issues_all_years_per_1000'].sort_values(ascending=False).head(10)

9807001    42556.0
9818001    21061.0
0303003    19479.0
0701018    10662.0
9811003     5154.0
0102034     3091.0
0107021     2940.0
0612002     2455.0
1101033     2435.0
0512001     2291.0
Name: total_issues_all_years_per_1000, dtype: float64

## Conclusion

Since the same 5 blocks groups show up year after year as having higher density than the others, and bc that's bc they're in low-population areas, they're outliers. Let's remove them.